In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *

import torch
from torch import nn
from dsin.ae.data_manager.data_loader import (
    SideinformationImageImageList, ImageSiTuple)
from dsin.ae import config
from dsin.ae.base_ae import BaseAutoEncoder
from dsin.ae.si_autoenc import SideInformationAutoEncoder

from dsin.ae.si_net import SiNetChannelIn
from dsin.ae.loss_man import LossManager
from dsin.ae.distortions import Distortions, DistTypes
from dsin.ae.kitti_normalizer import ChangeImageStatsToKitti, ChangeState
from dsin.ae import config


In [3]:
config.H_target = 2* 0.3
# config.autoencoder_loss_distortion_to_minimize=DistTypes.MS_SSMIM
config.K_MS_SSIM=500
config.beta

500

In [4]:
import time
import datetime
import logging

logger = logging.getLogger()

def setup_file_logger(log_file='out.log'):
    hdlr = logging.FileHandler(log_file)
    formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
    hdlr.setFormatter(formatter)
    logger.addHandler(hdlr) 
    logger.setLevel(logging.INFO)
    logger.info("start")

setup_file_logger()

In [5]:
class AverageMetric(Callback):
    "Wrap a `func` in a callback for metrics computation."
    def __init__(self, func,name):
        # If it's a partial, use func.func
        #         name = getattr(func,'func',func).__name__
        self.func, self.name = func, name

    def on_epoch_begin(self, **kwargs):
        "Set the inner value to 0."
        self.val, self.count = 0.,0

    def on_batch_end(self, last_output, last_target, **kwargs):
        "Update metric computation with `last_output` and `last_target`."
        if not is_listy(last_target): last_target=[last_target]
        self.count += last_target[0].size(0) # batch size
        
        if last_output[0] is None:
            X_OUTPUT = 1 # X_DEC
        else:
            X_OUTPUT = 0 # X_HAT_OUT
        val = self.func(last_output[X_OUTPUT], last_target[0])
        self.val += last_target[0].size(0) * val.detach().cpu()
        
    
    
    def on_epoch_end(self, last_metrics, **kwargs):
        "Set the final result in `last_metrics`."
        return add_metrics(last_metrics, self.val/self.count)
    
class ParameterMetricCallback(Callback):
    def __init__(self,loss_man):
        self.loss_man = loss_man
    
    def on_backward_begin(self,*args, **kwargs):
        self.pbar=kwargs["pbar"]
        if hasattr(self.loss_man,'soft_bit_entropy'):
            self.pbar.child.comment += f' soft_bit_entropy: {self.loss_man.soft_bit_entropy:.4f}'

class ParameterRunningAverageMetricCallback(Callback):
    def __init__(self,loss_man,alpha=0.1):
        self.loss_man = loss_man
        self.alpha = alpha
        self.val = None
    
    def on_backward_begin(self,*args, **kwargs):
        self.pbar=kwargs["pbar"]
        self.importance_map=kwargs["last_output"][3].detach()
#         import pdb
#         pdb.set_trace()
        if hasattr(self.loss_man,'soft_bit_entropy'):
            if self.val is None:
                self.val = self.loss_man.soft_bit_entropy.detach()
            else:
                self.val *= 1 - self.alpha
                self.val += self.alpha * self.loss_man.soft_bit_entropy.detach()
                
            self.pbar.child.comment += f' avg_bpp: {self.val / 2 :.4f} imp-mean-var {(torch.mean(self.importance_map),torch.var(self.importance_map))}'
            msg = f"bitcost_loss={self.loss_man.bit_cost_loss_value:.1f} "
            msg += f"l2reg_loss={self.loss_man.l2_reg_loss:.1f} "
            msg += f"autoencoder_loss_value={ self.loss_man.autoencoder_loss_value:.1f} "
            msg += f"si_loss={self.loss_man.si_net_loss_value} "
            self.pbar.child.comment += msg
    


In [6]:
class BitEntropy(Callback):
    "Wrap a `func` in a callback for metrics computation."
    def __init__(self,loss_man,alpha=0.1, logger=logger):
        # If it's a partial, use func.func
        #         name = getattr(func,'func',func).__name__
        self.loss_man = loss_man
        self.alpha = alpha
        self.logger = logger
        
    def on_epoch_begin(self, **kwargs):
        "Set the inner value to 0."
        self.val = 0.0
        self.iter = 0
        
        self.val_si_loss = None
        
    def on_batch_end(self, last_output, last_target, **kwargs):
        "Update metric computation with `last_output` and `last_target`."
      
        self.val *= 1 - self.alpha
        self.val += self.alpha * self.loss_man.soft_bit_entropy.detach()
        if self.val_si_loss is None:
            self.val_si_loss = self.loss_man.si_net_loss_value
        else :
            self.val_si_loss *= 1 - self.alpha
            self.val_si_loss += self.alpha * self.loss_man.si_net_loss_value.detach()
            
        if self.iter % 500 == 0 :
            importance_map  = last_output[3].detach()
            msg = f"iter {self.iter}: bpp = {self.val / 2:.3f}, impmap- mean {torch.mean(importance_map):.4f} var {torch.var(importance_map):.4f} "
            msg += f" total loss{self.loss_man.total_loss:.1f}  l2reg_loss={self.loss_man.l2_reg_loss:.1f}"
            msg += f"autoencoder_loss_value={ self.loss_man.autoencoder_loss_value:.1f}"
            msg += f"si_loss={self.val_si_loss}"
            self.logger.info(msg)
            print(msg)
        self.iter += 1

    def on_epoch_end(self, last_metrics, **kwargs):
        "Set the final result in `last_metrics`."
        return add_metrics(last_metrics, self.val)
    

In [7]:
config.use_si_flag = SiNetChannelIn.WithSideInformation
config.use_si_flag

<SiNetChannelIn.WithSideInformation: 6>

In [8]:
torch.__version__

'1.5.1'

In [9]:
# x = nn.Sequential(nn.Conv2d(in_channels=2,out_channels=3 , kernel_size=1),nn.ReLU())

#  LOAD BASE-AE

In [10]:
baseline_autoencoder = BaseAutoEncoder()
path = "../src/dsin/data"
pct= 1 #0.0005 #0.25

valid_image_list = SideinformationImageImageList.from_csv(
    path=path, csv_names=["KITTI_stereo_val.txt"],pct=pct)
train_image_list = SideinformationImageImageList.from_csv(
    path=path, csv_names=["KITTI_stereo_train.txt"],pct=pct)

image_lists = ItemLists(
    path=path, train=train_image_list, valid=valid_image_list)


tfms =  get_transforms(do_flip=True, flip_vert=False, max_rotate=None, max_zoom=1., max_lighting=None, max_warp=None, p_affine=0.0, p_lighting=0.0)

batchsize = 1

data = (image_lists
        .label_from_func(lambda x: x)
        .transform(None, size=(336, 1224), resize_method=ResizeMethod.CROP, tfm_y=True)
        .databunch(bs=batchsize))

learn = Learner(data=data,
                     model=baseline_autoencoder,
                     opt_func=torch.optim.Adam,
                     loss_func=LossManager(SiNetChannelIn.NoSideInformation),
                     metrics=[AverageMetric(Distortions._calc_dist,"MS_SSIM")])
learn.load('200815MAE-l2reg-baseline-1')


Learner(data=ImageDataBunch;

Train: LabelList (1576 items)
x: SideinformationImageImageList
ImageSiTuple [tensor([[[0.0763, 0.0793, 0.0785,  ..., 1.0000, 1.0000, 1.0000],
         [0.0766, 0.0775, 0.0755,  ..., 1.0000, 1.0000, 1.0000],
         [0.0518, 0.0547, 0.0576,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [0.1078, 0.1076, 0.0978,  ..., 0.0774, 0.0630, 0.0559],
         [0.1098, 0.1160, 0.1369,  ..., 0.0546, 0.0509, 0.0490],
         [0.1007, 0.1306, 0.1882,  ..., 0.0350, 0.0380, 0.0410]],

        [[0.0965, 0.0942, 0.0775,  ..., 1.0000, 1.0000, 1.0000],
         [0.0736, 0.0775, 0.0726,  ..., 1.0000, 1.0000, 1.0000],
         [0.0608, 0.0638, 0.0668,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [0.1381, 0.1401, 0.1330,  ..., 0.0729, 0.0756, 0.0766],
         [0.1501, 0.1529, 0.1425,  ..., 0.0548, 0.0557, 0.0558],
         [0.1892, 0.2082, 0.2097,  ..., 0.0418, 0.0449, 0.0480]],

        [[0.0676, 0.0347, 0.0333,  ..., 1.0000, 1.0000, 1.0000],
         [0.0677

In [11]:
learn.layer_groups

[Sequential(
   (0): ChangeImageStatsToKitti()
   (1): Conv2d(3, 64, kernel_size=[5, 5], stride=[2, 2], padding=[2, 2], padding_mode=replicate)
   (2): ReLU()
   (3): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
   (4): Conv2d(64, 128, kernel_size=[5, 5], stride=[2, 2], padding=[2, 2], padding_mode=replicate)
   (5): ReLU()
   (6): BatchNorm2d(128, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
   (7): Conv2d(128, 128, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], padding_mode=replicate)
   (8): ReLU()
   (9): BatchNorm2d(128, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
   (10): Conv2d(128, 128, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], padding_mode=replicate)
   (11): Conv2d(128, 128, kernel_size=[3, 3], stride=[1, 1], padding=[1, 1], padding_mode=replicate)
   (12): ReLU()
   (13): BatchNorm2d(128, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
   (14): Conv2d(128, 128, kernel_size=

## Freeze Base AE before training first time with SI-AE

In [12]:
learn.freeze_to(-1)

# Create  SI-AE

In [13]:
si_autoencoder = SideInformationAutoEncoder(base_line_auto_enc = learn.model)
learn = Learner(data=data,
                     model=si_autoencoder,
                     opt_func=torch.optim.Adam,
                     loss_func=LossManager(SiNetChannelIn.WithSideInformation),
                     metrics=[AverageMetric(Distortions._calc_dist,"MS_SSIM")])

In [ ]:
model_fname = '200815MAE-l2reg-res-si'
for i in range(1,5):
    if i != 1 :
        learn.load(f'{model_fname}-{i - 1}',with_opt=True)
    
    config.si_loss_weight_alpha = 0.7
    learn.model.true_tuple_loss_false_just_out = True
    learn.model.use_side_infomation = SiNetChannelIn.WithSideInformation
    learn.loss_func.use_side_infomation = SiNetChannelIn.WithSideInformation
    learn.fit(1, lr=0.0001,wd=0,callbacks=[ParameterRunningAverageMetricCallback(learn.loss_func),BitEntropy(loss_man=learn.loss_func)])
    learn.save(f'{model_fname}-{i}')
    !aws s3 cp ~/tDSIN/src/dsin/data/models/{model_fname}-{i}.pth  s3://dsin-us/models/
    !aws s3 cp ~/tDSIN/tutorials/out.log s3://dsin-us/models/{model_fname}-{i}.log


epoch,train_loss,valid_loss,MS_SSIM,time


iter 0: bpp = 0.049, impmap- mean 0.3327 var 0.2220  total loss380.5  l2reg_loss=35.9autoencoder_loss_value=15.7si_loss=210.5915069580078
iter 500: bpp = 0.319, impmap- mean 0.1479 var 0.1260  total loss69.1  l2reg_loss=36.1autoencoder_loss_value=27.5si_loss=34.22037887573242
iter 1000: bpp = 0.315, impmap- mean 0.1499 var 0.1274  total loss71.9  l2reg_loss=36.1autoencoder_loss_value=28.3si_loss=31.401487350463867
iter 1500: bpp = 0.308, impmap- mean 0.1413 var 0.1213  total loss55.0  l2reg_loss=36.2autoencoder_loss_value=19.1si_loss=18.729248046875


In [ ]:
learn.model.my_tuple[5]


In [ ]:
learn.data.train_ds[1][0].img

In [ ]:
learn.data.train_ds[1][0].si_img

In [ ]:
learn.loss_func.soft_bit_entropy

In [ ]:
config.si_loss_weight_alpha

In [ ]:
learn.loss_func.autoencoder_loss_value

In [ ]:
mt = learn.model.my_tuple[-1].squeeze_().detach()
mx, mn =torch.max(mt), torch.min(mt)
diff = mx-mn
show_image(Image((mt - mn)/(diff)),figsize=(30,30))
# # print(mn)

In [ ]:

# # show_image(Image(learn.model.my_tuple[0][:,:50,:50]/255))
# # Image(learn.model.my_tuple[2].squeeze_().detach()/255.0)
mt = learn.model.my_tuple[2].squeeze_().detach()
mx, mn =torch.max(mt), torch.min(mt)
diff = mx-mn
show_image(Image((mt - mn)/(diff)),figsize=(30,30))
# # print(mn)

# torch.min(learn.model.my_tuple[2].squeeze_().detach())

In [ ]:
aaa = (mt - mn)/(diff)

flip_lr(Image(aaa.cpu()))

In [ ]:
# learn.model.true_tuple_loss_false_just_out = False

# learn.show_results(figsize=(30,30))

In [ ]:
learn.model.my_tuple[-2].data